In [2]:
# Load the file that we will be using
with open("dexter_transcript/season_1/season_1_transcript_filtered.txt", 'r') as f:
    file_content = f.read()

# Print the number of characters in the file
print(f"Characters in File : {len(file_content)}")

Characters in File : 4103650
